# Chapter 3. 허브

1. 중심성 측도 [중심도]
2. 중심도 분포  
3. robustness test


## 1. 허브 찾기

사람들은 종종 네트워크에서 가장 중요한 노드를 찾으려고 노력한다. 즉, 중심성의 가장 기본적인 측도는 **차수(degree)** 즉, 노드에 연결된 링크[엣지]의 수다.

Enron 임원 이메일 그래프를 살펴보자.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
G = nx.read_edgelist('../datasets/ia-enron-only/ia-enron-only.edges', nodetype=int)
#G = nx.read_edgelist('ia-enron-only.edges', nodetype=int)
print(G)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/ia-enron-only/ia-enron-only.edges'

In [ ]:
plt.figure(figsize=(10,10)) 
nx.draw(G, with_labels=True)
plt.show()

### 기본적으로 nx.read_edgelist는 노드 이름이 문자열이라고 가정한다.  
'엣지 목록(Edge lists)'은 그래프를 저장하기 위한 단순하고 일반 텍스트 형식이다.  
이 간단한 파일 형식에는 데이터 유형에 대한 정보가 포함되어 있지 않으므로 모든 노드 이름은 기본적으로 문자열로 간주된다.  
이 예에서와 같이 노드 이름이 정수로 제공되는 경우 노드 이름과의 혼동을 피하기 위해 'nodetype = int' 키워드 인수를 지정해야 한다.


### max 함수
가장 높은 차수를 가진 노드를 찾기 위해 Python의 내장 max 함수를 사용할 것이다.

In [ ]:
max([1,2,3,4,5])

그러나 **가장 큰** 항목이 항상 분명한 것은 아닙니다.

In [ ]:
max(['apple', 'grape', 'carrot'])

다른 옵션보다 **grape**가 더 큰 이유는 무엇인가?  
문자열의 기본 정렬은 사전순(기본적으로 알파벳순)이기 때문이다.  
이 기본 순서를 원하지 않으면, Python에 항목을 비교하는 방법을 알려주는 key 함수를 지정할 수 있다.

In [ ]:
max(['apple', 'grape', 'carrot'], key=len)

이제 'carrot'이 가장 큰 요소다. 항목을 길이로 비교하고 있기 때문이다.

### 1. 차수 기반 중심도: 최대 차수를 가진 노드 찾기

어떤 기준에 따라 '최대 노드'를 얻기 위해 max 함수를 적용할 수 있다.  
우리의 경우 노드를 그 **차수** 별로 비교하려고 한다.

In [ ]:
highest_degree_node = max(G.nodes, key=G.degree)
highest_degree_node

In [ ]:
G.degree(highest_degree_node)

key 함수인 `G.degree`에 전달될 때 가장 높은 값을 제공하는 `G.nodes`의 항목을 원한다고 `max` 함수에 알리고 있다.  
이 구조는 `G.degree`가 함수이기 때문에 작동한다.

### 2. 근접 중심도(Closeness Centrality)
* 참고 사이트: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.closeness_centrality.html  

노드의 중심도를 측정하는 또 다른 방법은 그 노드가 다른 노드와 얼마나 '가까운지'를 결정하는 것이다.  
이것은 한 노드에서 다른 모든 노드까지의 거리를 더해서 정한다.
거리가 평균적으로 짧으면 그 합한 양이 작을 것이고, 노드의 중심도가 높다고 할 수 있다.  

#### 근접 중심도는 한 노드에서 다른 모든 노드까지의 거리 합의 역수다.

In [ ]:
nx.closeness_centrality(G)   # 노드의 근접 중심도

### 3. 사이 중심도(betweenness centrality)
* 참고 사이트: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html

네트워크에서 발생하는 많은 현상은 **확산과정**을 바탕으로 한다.  
이 확산과정을 기반으로 하여 **사이 중심도**가 제안되었다.
**확산의 종류**에 따라 **사이 중심도**는 다르게 정의된다.  
가장 널리 사용되는 **사이 중심도**는 **최단 경로**를 따라 한 노드에서 다른 노드로 신호가 전달되는 간단한 과정을 고려한다.

betweenness_centrality 함수는 모든 노드의 중심성 값을 한 번에 생성하고 **딕셔너리**를 반환한다.  
이 경우 추가 단계가 필요하다.

In [ ]:
betweenness = nx.betweenness_centrality(G)
highest_betweenness_node = max(G.nodes, key=betweenness.get)
highest_betweenness_node

In [ ]:
betweenness[highest_betweenness_node]

## 2. 중심도 분포(Centrality distributions)

우리는 네트워크에서 가장 중심적인 노드를 찾았지만, 종종 모든 노드의 중심성에 대한 정보를 요약하려고 한다.  
예를 들어 **최대 차수** 외에도 네트워크에서 **평균과 중간 차수**를 원할 때가 많다.

이 정보를 요약하는 첫 번째 단계는 그래프에서 모든 중심성 값의 시퀀스를 얻는 것이다.  
우리는 더 이상 노드 이름에 신경 쓰지 않고 일련의 숫자만 원한다.  
**차수**의 예부터 시작해 보자.

In [ ]:
degree_sequence = [G.degree(n) for n in G.nodes]

이 시퀀스의 평균과 중앙값을 얻으려면 Python의 내장 statistics 모듈을 사용할 수 있다.

In [ ]:
import statistics

print('Mean degree:', statistics.mean(degree_sequence))
print('Median degree:', statistics.median(degree_sequence))

전체 네트워크에 대해 한 번에 계산되고 **딕셔너리**를 반환하는 다른 중심성 측도의 경우, **딕셔너리**의 **.values()** 를 사용할 수 있다.

In [ ]:
betweenness = nx.betweenness_centrality(G)
betweenness_sequence = list(betweenness.values())

print('Mean betweenness:', statistics.mean(betweenness_sequence))
print('Median betweenness:', statistics.median(betweenness_sequence))

### 분포 그림 그리기
히스토그램을 사용하여 중심성 값의 시퀀스를 그림으로 나타낼 수 있다.  
기본 형태에서 히스토그램은 x축에 차수 값을 표시하고 y축에 해당 차수를 갖는 노드 수를 표시한다.  
이 계산을 수행하기 위해 Python의 collections.Counter를 사용할 수 있다.

In [ ]:
from collections import Counter

degree_counts = Counter(degree_sequence)
degree_counts

이 기본 히스토그램 플롯에서 x값은 시퀀스의 최소도와 최대도 사이의 모든 정수다.  
degree_counts.keys()는 차수 시퀀스에서 볼 수 있는 모든 개별 값을 제공한다.  
또한 올바른 엔드포인트를 포함하고 싶기 때문에 아래 범위에서 +1이 필요하다.

In [ ]:
min_degree, max_degree = min(degree_counts.keys()), max(degree_counts.keys())

plot_x = list(range(min_degree, max_degree + 1))

우리의 y-값은 차수열에서 각 x-값을 세는 횟수다.  
시퀀스에 표시되지 않는 차수 값을 0으로 계산하기 위해, 기본값과 함께 .get 메서드를 사용할 수 있다.  
예를 들어 32는 위의 degree_counts에 표시되지 않으므로, degree_counts.get(32, 0)은 0을 제공한다.

In [ ]:
plot_y = [degree_counts.get(x, 0) for x in plot_x]

In [ ]:
plt.bar(plot_x, plot_y)
plt.xlabel('degree')
plt.ylabel('count')
plt.show()

### Histogram binning


편안하게 표시할 수 있는 것보다 더 많은 x 값이 있거나 사이 중심성의 경우처럼 중심성 측정이 불연속적이지 않은 경우 Histogram binning을 사용할 수 있다.  
이것은 bin이라고 하는 일련의 분리된 간격을 정의하고 이러한 각 bin에 속하는 값의 수를 계산한다.  
가장 간단한 경우에 pyplot의 hist 함수에 원하는 bin 수를 알려주면 binning이 자동으로 수행된다.

In [ ]:
counts, bins, patches = plt.hist(betweenness_sequence, bins=10)
plt.xlabel('bins')
plt.ylabel('counts')
plt.show()

In [ ]:
bins

In [ ]:
counts

이를 통해 0에서 0.0194006 사이에 115개의 값, 0.194006에서 0.3880121 사이에 13개의 값 등이 있음을 알 수 있다.

## 3. Testing robustness

네트워크에서 노드의 **상대적 중요성**에 대해 생각하는 또 다른 방법은 특정 노드가 제거될 경우 네트워크 구조가 얼마나 손상되는지 측정하는 것이다.  
실생활에서 노드 제거는 사람이 소셜 네트워크에서 멀어져서 나가고, 누군가 직업을 바꾸고, 이메일 네트워크에서 제거되고, 인터넷 라우터가 공격/과부화되어 다운되는 등일 수 있다.

광범위하게 우리는 **무작위 실패(random failure)** 와 **표적 공격(targeted attack)** 이라는 두 가지 유형의 네트워크 손상을 고려한다.  
무작위 실패에서는 제거할 노드가 무작위로 선택된다.  
표적 공격에서는 몇 가지 기준에 따라 노드를 제거한다. 예를 들어 중심성 정도가 내림차순으로 노드를 제거한다.

### 연결된 구성요소(Connected components)

**손상(damage)** 을 측정하기 위해 때때로 **코어(core)** 라고 하는 네트워크의 가장 큰 연결 구성요소의 크기를 측정한다.  
먼저 nx.connected_components가 가장 큰 것부터 시작하여 한 번에 하나씩 연결된 구성요소를 제공하는 생성기(generator)임을 관찰한다.

In [ ]:
nx.connected_components(G)

우리는 종종 **코어(core)** 또는 **가장 큰 연결 구성요소**만 원하기 때문에 next 함수를 사용하여 생성기에서 첫 번째 항목만 가져올 수 있다.  
각 구성요소는 노드 이름 세트로 제공된다.

In [ ]:
core = next(nx.connected_components(G))
core

따라서 이 집합의 len은 이 구성요소의 노드 수를 제공한다.

In [ ]:
len(core)

연결된 모든 구성요소를 원하면 그 구성요소들의 list를 얻을 수 있다.

In [ ]:
components = list(nx.connected_components(G))

이 list의 길이는 연결된 구성요소의 수다.

In [ ]:
len(components)

### 무작위 실패(Random Failure)

파괴적인 프로세스에 관여할 때마다 공격할 네트워크 그래프의 복사본을 만들어 쉽게 원래 상태로 되돌릴 수 있다.

In [ ]:
C = G.copy()

**무작위 실패**를 시뮬레이트하기 위해 일부 노드 이름을 무작위로 선택하고 그래프에서 제거한다.  
random.sample을 사용하여 한 번에 둘 이상의 노드를 제거할 수 있다.  
무작위로 샘플링할 노드 이름 list를 만들어야 한다.

In [ ]:
import random

nodes_to_remove = random.sample(list(C.nodes), 2)
C.remove_nodes_from(nodes_to_remove)

전체 시뮬레이션은 네트워크의 새로운 사본에서 시작하여 다음과 같이 작동한다.

1. 원래 네트워크 크기와 비교하여 네트워크 코어 크기를 측정한다.
2. 임의로 M 노드를 선택하고 제거한다.
3. 노드가 M개 미만이 될 때까지 반복한다.

우리는 이 프로세스를 수행하기를 원하는 단계 수에서 M을 결정할 것이다.  
약 25단계가 적당하므로 다음과 같다.

In [ ]:
G.number_of_nodes()

In [ ]:
number_of_steps = 25
M = G.number_of_nodes() // number_of_steps   # //: floor division
M

그런 다음 range를 사용하여 각 단계에서 제거된 총 노드 수의 시퀀스를 생성할 수 있다.

In [ ]:
num_nodes_removed = range(0, G.number_of_nodes(), M)

loop는 매우 간단하다. 각 단계에서 코어에 남아 있는 노드의 비율을 기록해야 한다.

In [ ]:
N = G.number_of_nodes()
C = G.copy()
random_attack_core_proportions = []
for nodes_removed in num_nodes_removed:
    # Measure the relative size of the network core
    core = next(nx.connected_components(C))
    core_proportion = len(core) / N
    random_attack_core_proportions.append(core_proportion)
    
    # If there are more than M nodes, select M nodes at random and remove them
    if C.number_of_nodes() > M:
        nodes_to_remove = random.sample(list(C.nodes), M)
        C.remove_nodes_from(nodes_to_remove)

In [ ]:
plt.title('Random failure')
plt.xlabel('Number of nodes removed')
plt.ylabel('Proportion of nodes in core')
plt.plot(num_nodes_removed, random_attack_core_proportions, marker='o')
plt.show()

#### 표적 공격(Targeted Attack)

표적 공격 시뮬레이션은 무작위로 선택하는 대신 각 단계에서 가장 중앙에 있는 M개의 노드를 선택한다는 점을 제외하면 비슷하다.  
이를 달성하기 위해 가장 중앙 노드를 얻기 위해 초기에 사용된 max 함수와 같은 것을 원하지만 최상위 M 노드를 얻을 수 있다.  
Python의 sorted 함수를 max와 유사한 방식으로 사용하여 우선 내림차순 또는 역순으로 중심성을 기준으로 노드를 정렬할 수 있다.  
**차수** 별로 정렬되면 목록에서 첫 번째 M 노드를 가져온다.

In [ ]:
nodes_sorted_by_degree = sorted(G.nodes, key=G.degree, reverse=True)
top_degree_nodes = nodes_sorted_by_degree[:M]
top_degree_nodes

In [ ]:
N = G.number_of_nodes()
number_of_steps = 25
M = N // number_of_steps

num_nodes_removed = range(0, N, M)
C = G.copy()
targeted_attack_core_proportions = []
for nodes_removed in num_nodes_removed:
    # Measure the relative size of the network core
    core = next(nx.connected_components(C))
    core_proportion = len(core) / N
    targeted_attack_core_proportions.append(core_proportion)
    
    # If there are more than M nodes, select top M nodes and remove them
    if C.number_of_nodes() > M:
        nodes_sorted_by_degree = sorted(C.nodes, key=C.degree, reverse=True)
        nodes_to_remove = nodes_sorted_by_degree[:M]
        C.remove_nodes_from(nodes_to_remove)

In [ ]:
plt.title('Targeted attack')
plt.xlabel('Number of nodes removed')
plt.ylabel('Proportion of nodes in core')
plt.plot(num_nodes_removed, targeted_attack_core_proportions, marker='o')

보시다시피 그 효과는 극적이다. 그래프에서 상대적으로 적은 수의 중앙 노드를 제거한 후 네트워크가 완전히 연결 해제된다.

pyplot은 추가 효과를 위해 동일한 플롯에 이러한 곡선을 그릴 수 있다.

In [ ]:
plt.title('Random failure vs. targeted attack')
plt.xlabel('Number of nodes removed')
plt.ylabel('Proportion of nodes in core')
plt.plot(num_nodes_removed, random_attack_core_proportions, marker='o', label='Failures')
plt.plot(num_nodes_removed, targeted_attack_core_proportions, marker='^', label='Attacks')
plt.legend()